## LLaMa 3.1

LLaMa 3.1 is a year old model from Meta. Check out [the model card](https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct) for further details. It is open-sourced.  To use it, you need to log in to your Hugging Face account and get permission.  We're using the 8 billion parameter version but quantized so it has a much smaller memory footprint.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datasci-w266/2025-summer-main/blob/master/materials/lesson_notebooks/lesson_7_summarization_LLM.ipynb)

In [1]:
!pip install -q -U bitsandbytes flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 38.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.4 MB/s eta 0:00:00


In [2]:
import torch
from transformers import pipeline
from pprint import pprint

Here's some text from the introduction to [The Prompt Report: A Systematic Survey of Prompting Techniques](https://arxiv.org/pdf/2406.06608).  Let's have the model summarize it.

In [3]:
ARTICLE = "Scope of Study We create a broad directory of prompting techniques, which can be quickly understood and easily implemented for rapid experimentation by developers and researchers. To this end, we limit our study to focus on discrete prefix prompts (Shin et al., 2020a) rather than cloze prompts (Petroni et al., 2019; Cui et al., 2021), because modern LLM architectures (especially decoder-only models), which use prefix prompts, are widely used and have robust support for both consumers and researchers. Additionally, we refined our focus to hard (discrete) prompts rather than soft (continuous) prompts and leave out papers that make use of techniques using gradient-based updates (i.e. fine-tuning). Finally, we only study task-agnostic techniques. These decisions keep the work approachable to less technical readers and maintain a manageable scope. "

ARTICLE += "Sections Overview We conducted a machine-assisted systematic review grounded in the PRISMA process (Page et al., 2021) (Section 2.1) to identify 58 different text-based prompting techniques, from which we create a taxonomy with a robust terminology of prompting terms (Section 1.2) While much literature on prompting focuses on English-only settings, we also discuss multilingual techniques (Section 3.1). Given the rapid growth in multimodal prompting, where prompts may include media such as images, we also expand our scope to multimodal techniques (Section 3.2). Many multilingual and multimodal prompting techniques are direct extensions of English text-only prompting techniques. "

ARTICLE += "As prompting techniques grow more complex, they have begun to incorporate external tools, such as Internet browsing and calculators. We use the term ‘agents‘ to describe these types of prompting techniques (Section 4.1). It is important to understand how to evaluate the outputs of agents and prompting techniques to ensure accuracy and avoid hallucinations."

len(ARTICLE)

1899

In [4]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,


)

Let's load the LLama 3.1 model using the Pipeline abstraction from Hugging Face and before we have it summarize, let's ask it a question and see how well it answers.  Do you think the answer is accurate?  How might we evaluate the generated answer?

In [5]:
#import transformers
#import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16, "quantization_config": quantization_config},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a science communicator who makes technology accessible to everyone!"},
    {"role": "user", "content": "Please write a five sentence explanation of how LLMs do knowledge representation."},
]

outputs = pipeline(
    messages,
    max_new_tokens=512,
)

pprint(outputs[0]["generated_text"][-1], compact=True)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'content': 'Large Language Models (LLMs) like myself represent knowledge '
            'through complex neural networks that process and store vast '
            'amounts of information in a distributed manner. At the heart of '
            'this process lies a type of knowledge representation known as a '
            '"knowledge graph," where interconnected nodes and edges capture '
            "relationships between concepts, entities, and ideas. When I'm "
            'trained on a dataset, I build a graph-like structure where each '
            'node represents a word or concept, and the edges between them '
            'signify the semantic connections between these nodes, such as '
            'synonyms, antonyms, or co-occurrences. This knowledge graph is '
            'then used to retrieve and generate text based on patterns and '
            'associations learned from the training data. By leveraging this '
            'knowledge graph, I can draw inferences, answer question

Now let's try it for abstractive summarization.  Note that it takes a lot longer to generate answers because this model has 8 billion.  The next cell can take up to 2 minutes to complete.

How good is the output from Llama3.1?  How can we measure the performance? What's are all of the elements we need to say run ROUGE?

In [6]:
messages = [
            {"role": "system", "content": "You are an expert on natural language processing.  Please summarize the following content for a fifth grader. Your summary should be no longer than five sentences."},
            {"role": "user", "content": ARTICLE},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

#lets set some values to have more control over the output
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
pprint(outputs[0]["generated_text"][len(prompt):], compact=True)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


('Imagine you have a super smart computer program that can understand and '
 'respond to questions or tasks. To help people use these programs better, '
 'researchers are studying different ways to give them instructions, called '
 '"prompts." They\'re focusing on simple, specific instructions, like "What\'s '
 'the capital of France?" rather than more complex instructions that might '
 "need to be adjusted as they're used. They're also looking at how to use "
 'these prompts with different languages and with pictures or other types of '
 "media. To make it easier for people to understand, they're breaking down the "
 'different techniques into categories and explaining them in a clear way.')


Try it yourself.  You can fill in the system and the user portion of the prompt.  See what kinds of questions it can answer and see how well it summarizes content.

In [7]:
messages = [
            {"role": "system", "content": "Your Value Here"},
            {"role": "user", "content": "Your Value Here"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

#lets set some values to have more control over the output
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
pprint(outputs[0]["generated_text"][len(prompt):], compact=True)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


('It seems you just repeated the phrase. Could you provide more context or '
 'clarify what you mean by "Your Value Here"?')
